In [115]:
import cv2
import numpy as np

In [116]:
lena=cv2.imread("../assets/Lena.jpg", 0)

cv2.imshow("lena", lena)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [117]:
def gaussianFunction(x,y,sigma):
    return (1/(2*np.pi*sigma**2))*np.exp(-(x**2 + y**2) /(2*sigma**2))

In [118]:
def gaussianKernel_x(size, sigma):

    k = size // 2
    coords = np.arange(-k, k + 1)
    x,y = np.meshgrid(coords, coords)
    gaussval = gaussianFunction(x, y, sigma)
    kernel = -(x / sigma**2) * gaussval
    kernel /= np.sum(kernel)

    return kernel

def gaussianKernel_y(size, sigma):
   
    k = size // 2
    coords = np.arange(-k, k + 1)
    x,y = np.meshgrid(coords, coords)
    gaussval = gaussianFunction(x, y, sigma)
    kernel = -(y / sigma**2) * gaussval
    kernel /= np.sum(kernel)

    return kernel
            

gaussian_kernel_x=gaussianKernel_x(7, 1)
gaussian_kernel_y=gaussianKernel_y(7, 1)


print(gaussian_kernel_x)


gaussian_kernel_x=gaussianKernel_x(7, 1)
gaussian_kernel_y=gaussianKernel_y(7, 1)


print(gaussian_kernel_x)

print(gaussian_kernel_y)

[[-2.79691995e+12 -2.27156403e+13 -5.09022183e+13  0.00000000e+00
   5.09022183e+13  2.27156403e+13  2.79691995e+12]
 [-3.40734604e+13 -2.76733150e+14 -6.20115968e+14  0.00000000e+00
   6.20115968e+14  2.76733150e+14  3.40734604e+13]
 [-1.52706655e+14 -1.24023194e+15 -2.77916695e+15  0.00000000e+00
   2.77916695e+15  1.24023194e+15  1.52706655e+14]
 [-2.51770710e+14 -2.04479677e+15 -4.58207167e+15  0.00000000e+00
   4.58207167e+15  2.04479677e+15  2.51770710e+14]
 [-1.52706655e+14 -1.24023194e+15 -2.77916695e+15  0.00000000e+00
   2.77916695e+15  1.24023194e+15  1.52706655e+14]
 [-3.40734604e+13 -2.76733150e+14 -6.20115968e+14  0.00000000e+00
   6.20115968e+14  2.76733150e+14  3.40734604e+13]
 [-2.79691995e+12 -2.27156403e+13 -5.09022183e+13  0.00000000e+00
   5.09022183e+13  2.27156403e+13  2.79691995e+12]]
[[-2.79691995e+12 -2.27156403e+13 -5.09022183e+13  0.00000000e+00
   5.09022183e+13  2.27156403e+13  2.79691995e+12]
 [-3.40734604e+13 -2.76733150e+14 -6.20115968e+14  0.00000000e+

In [119]:
def int_kernel(kernel, min_val, max_val):

    k_min = np.min(kernel)
    k_max = np.max(kernel)
    norm_kernel = (kernel - k_min) / (k_max - k_min) 
    int_kernel = norm_kernel * (max_val - min_val) + min_val
    int_kernel = np.round(int_kernel).astype(int)
    return int_kernel

x_int = int_kernel(gaussian_kernel_x, -25, 25)
y_int = int_kernel(gaussian_kernel_y, -25, 25)


print("Integer Kernel X:\n", x_int)
print("Integer Kernel Y:\n", y_int)

print("Sum of Integer Kernel X:", np.sum(x_int))
print("Sum of Integer Kernel Y:", np.sum(y_int))

Integer Kernel X:
 [[  0   0   0   0   0   0   0]
 [  0  -2  -3   0   3   2   0]
 [ -1  -7 -15   0  15   7   1]
 [ -1 -11 -25   0  25  11   1]
 [ -1  -7 -15   0  15   7   1]
 [  0  -2  -3   0   3   2   0]
 [  0   0   0   0   0   0   0]]
Integer Kernel Y:
 [[  0   0   1   1   1   0   0]
 [  0   2   7  11   7   2   0]
 [  0   3  15  25  15   3   0]
 [  0   0   0   0   0   0   0]
 [  0  -3 -15 -25 -15  -3   0]
 [  0  -2  -7 -11  -7  -2   0]
 [  0   0  -1  -1  -1   0   0]]
Sum of Integer Kernel X: 0
Sum of Integer Kernel Y: 0


In [120]:
def convolve2D(image : np.array ,  kernel : np.array) -> np.array:
    kernel = np.flip(kernel)
    ih, iw = image.shape
    kh , kw = kernel.shape
    border_size = kh//2
    img_bordered = cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT)
    bh,bw = img_bordered.shape
    result = np.zeros((ih, iw), dtype=np.float32)
    for i in range(ih):
        for j in range(iw):
            region = img_bordered[i:i+kh ,j:j+kw]
            mul = np.multiply(region,kernel)
            val = np.sum(mul)
            result[i,j] = val  
    return result 

In [121]:
convo_x = convolve2D(lena, gaussian_kernel_x)
norm_x = np.round(cv2.normalize(convo_x, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [122]:
cv2.imshow("Convo_X", convo_x)
cv2.imshow("norm_x", norm_x)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [123]:
convo_y = convolve2D(lena, gaussian_kernel_y)
norm_y = np.round(cv2.normalize(convo_y, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [124]:
cv2.imshow("Convo_y", convo_y)
cv2.imshow("norm_y", norm_y)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [125]:
gradient_mag = cv2.magnitude(convo_x.astype(np.float32), convo_y.astype(np.float32) )

grad_norm = np.round(cv2.normalize(gradient_mag, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

cv2.imshow("Grad", gradient_mag)
cv2.imshow("Grad Norm", grad_norm)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [126]:
threshold_value=50
_, binary_edges = cv2.threshold(grad_norm, threshold_value, 255, cv2.THRESH_BINARY)

cv2.imshow("cv2 threshold", binary_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [127]:
def double_threshold(image, low_thresh, high_thresh):
    output = np.zeros_like(image, dtype=np.uint8)
    output[image>=high_thresh]=255
    output[image<low_thresh]=0
    mask =(image >=low_thresh)&(image<high_thresh)
    output[mask] = 128

    return output
    

In [128]:
double_t = double_threshold(grad_norm, 50, 150)

cv2.imshow("double threshold threshold", double_t)
cv2.waitKey(0)
cv2.destroyAllWindows()